# Challenge 2: Sentiment Analysis

In this challenge we will learn sentiment analysis and practice performing sentiment analysis on Twitter tweets. 

## Introduction

Sentiment analysis is to *systematically identify, extract, quantify, and study affective states and subjective information* based on texts ([reference](https://en.wikipedia.org/wiki/Sentiment_analysis)). In simple words, it's to understand whether a person is happy or unhappy in producing the piece of text. Why we (or rather, companies) care about sentiment in texts? It's because by understanding the sentiments in texts, we will be able to know if our customers are happy or unhappy about our products and services. If they are unhappy, the subsequent action is to figure out what have caused the unhappiness and make improvements.

Basic sentiment analysis only understands the *positive* or *negative* (sometimes *neutral* too) polarities of the sentiment. More advanced sentiment analysis will also consider dimensions such as agreement, subjectivity, confidence, irony, and so on. In this challenge we will conduct the basic positive vs negative sentiment analysis based on real Twitter tweets.

NLTK comes with a [sentiment analysis package](https://www.nltk.org/api/nltk.sentiment.html). This package is great for dummies to perform sentiment analysis because it requires only the textual data to make predictions. For example:

```python
>>> from nltk.sentiment.vader import SentimentIntensityAnalyzer
>>> txt = "Ironhack is a Global Tech School ranked num 2 worldwide.   Our mission is to help people transform their careers and join a thriving community of tech professionals that love what they do."
>>> analyzer = SentimentIntensityAnalyzer()
>>> analyzer.polarity_scores(txt)
{'neg': 0.0, 'neu': 0.741, 'pos': 0.259, 'compound': 0.8442}
```

In this challenge, however, you will not use NLTK's sentiment analysis package because in your Machine Learning training in the past 2 weeks you have learned how to make predictions more accurate than that. The [tweets data](https://www.kaggle.com/kazanova/sentiment140) we will be using today are already coded for the positive/negative sentiment. You will be able to use the Naïve Bayes classifier you learned in the lesson to predict the sentiment of tweets based on the labels.

## Conducting Sentiment Analysis

### Loading and Exploring Data

The dataset we'll be using today is located on Kaggle (https://www.kaggle.com/kazanova/sentiment140). Once you have downloaded and imported the dataset, it you will need to define the columns names: df.columns = ['target','id','date','flag','user','text']

*Notes:* 

* The dataset is huuuuge (1.6m tweets). When you develop your data analysis codes, you can sample a subset of the data (e.g. 20k records) so that you will save a lot of time when you test your codes.

In [33]:
#install Kaggle API
#!conda install kaggle

In [34]:
import pandas as pd
#from kaggle.api.kaggle_api_extended import KaggleApi
import pandas
import numpy as np
from nltk.tokenize import word_tokenize
from functools import reduce
from nltk.stem import PorterStemmer

In [35]:
columns = ['target','id','date','flag','user','text']
df = pandas.read_csv(r"..\data\noemoticon.csv", encoding = "ISO-8859-1", header=None, names=columns)
display(df.shape, df.head())

(1600000, 6)

,target,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [36]:
df = df.drop(columns=["id", "date", "flag", "user"])
display(df.shape, df.head())

(1600000, 2)

,target,text
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,is upset that he can't update his Facebook by ...
2,0,@Kenichan I dived many times for the ball. Man...
3,0,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all...."


In [37]:
max_subset_size = 20000
df = df.sample(n=min(max_subset_size, len(df)))
display(df.shape, df.head())

(20000, 2)

,target,text
113994,0,Good morning everyone. It seems as though we h...
661328,0,@jakuba16 I wish I saw them when they were all...
237306,0,um....still not packed...friends here to move ...
1506861,4,@romzkeepomski just always do the RIGHT thing.
411814,0,@A__W ohh i miss you too what happened? i'll ...


In [38]:
df.reset_index()

,index,target,text
0,113994,0,Good morning everyone. It seems as though we h...
1,661328,0,@jakuba16 I wish I saw them when they were all...
2,237306,0,um....still not packed...friends here to move ...
3,1506861,4,@romzkeepomski just always do the RIGHT thing.
4,411814,0,@A__W ohh i miss you too what happened? i'll ...
...,...,...,...
19995,272810,0,back in VA = back to work
19996,1596899,4,@meeshly the only place to live!
19997,14829,0,@ddlovato i wish you would reply to mee...
19998,567131,0,That beer I ordered @ trivia night did NOT sta...


### Prepare Textual Data for Sentiment Analysis

Now, apply the functions you have written in Challenge 1 to your whole data set. These functions include:

* `clean_up()`

* `tokenize()`

* `stem_and_lemmatize()`

* `remove_stopwords()`

Create a new column called `text_processed` in the dataframe to contain the processed data. At the end, your `text_processed` column should contain lists of word tokens that are cleaned up. Your data should look like below:

![Processed Data](data-cleaning-results.png)

In [39]:
# your code here
import functions as fy

In [40]:
df['text_processed'] = df['text'].apply(fy.clean_up)
df['text_processed'] = df['text_processed'].apply(fy.tokenize)
df['text_processed'] = df['text_processed'].apply(fy.stem_and_lemmatize)
df['text_processed'] = df['text_processed'].apply(fy.remove_stopwords)
df

,target,text,text_processed
113994,0,Good morning everyone. It seems as though we h...,"[good, morn, everyon, seem, though, sad, turn,..."
661328,0,@jakuba16 I wish I saw them when they were all...,"[jakuba, wish, saw, togeth, ani, news, big]"
237306,0,um....still not packed...friends here to move ...,"[umstil, packedfriend, move, hour, uhoh]"
1506861,4,@romzkeepomski just always do the RIGHT thing.,"[romzkeepomski, alway, right, thing]"
411814,0,@A__W ohh i miss you too what happened? i'll ...,"[aw, ohh, miss, happen, ill, ill, see, day]"
...,...,...,...
272810,0,back in VA = back to work,"[back, va, back, work]"
1596899,4,@meeshly the only place to live!,"[meeshli, onli, place, live]"
14829,0,@ddlovato i wish you would reply to mee...,"[ddlovato, wish, would, repli, mee]"
567131,0,That beer I ordered @ trivia night did NOT sta...,"[beer, order, trivia, night, state, wa, oz, le..."


### Creating Bag of Words

The purpose of this step is to create a [bag of words](https://en.wikipedia.org/wiki/Bag-of-words_model) from the processed data. The bag of words contains all the unique words in your whole text body (a.k.a. *corpus*) with the number of occurrence of each word. It will allow you to understand which words are the most important features across the whole corpus.

Also, you can imagine you will have a massive set of words. The less important words (i.e. those of very low number of occurrence) do not contribute much to the sentiment. Therefore, you only need to use the most important words to build your feature set in the next step. In our case, we will use the top 5,000 words with the highest frequency to build the features.

In the cell below, combine all the words in `text_processed` and calculate the frequency distribution of all words. A convenient library to calculate the term frequency distribution is NLTK's `FreqDist` class ([documentation](https://www.nltk.org/api/nltk.html#module-nltk.probability)). Then select the top 5,000 words from the frequency distribution.

In [41]:
# your code here
from nltk import FreqDist

In [42]:
all_words = [word for tokens in df['text_processed'] for word in tokens]

# Calculate the frequency distribution of all words
freq_dist = FreqDist(all_words)

# Select the top 5,000 words with the highest frequency
top_5000_words = [word for word, _ in freq_dist.most_common(5000)]

In [43]:
top_5000_words

['im',
 'get',
 'go',
 'day',
 'wa',
 'thi',
 'good',
 'love',
 'like',
 'work',
 'today',
 'dont',
 'time',
 'think',
 'cant',
 'know',
 'miss',
 'see',
 'thank',
 'want',
 'make',
 'lol',
 'u',
 'back',
 'one',
 'realli',
 'feel',
 'hope',
 'night',
 'need',
 'well',
 'come',
 'still',
 'na',
 'watch',
 'amp',
 'look',
 'oh',
 'home',
 'new',
 'much',
 'ha',
 'twitter',
 'morn',
 'last',
 'great',
 'wish',
 'say',
 'sleep',
 'tomorrow',
 'ill',
 'wait',
 'sad',
 'take',
 'fun',
 'haha',
 'onli',
 'happi',
 'didnt',
 'tonight',
 'week',
 'thing',
 'friend',
 'bad',
 'tri',
 'nice',
 'veri',
 'would',
 'whi',
 'find',
 'follow',
 'sorri',
 'way',
 'hi',
 'right',
 'gon',
 'tweet',
 'though',
 'could',
 'ive',
 'school',
 'start',
 'better',
 'yeah',
 'play',
 'guy',
 'peopl',
 'even',
 'show',
 'use',
 'hate',
 'bed',
 'ye',
 'next',
 'hour',
 'final',
 'weekend',
 'girl',
 'awesom',
 'hey',
 'soon',
 'eat',
 'wan',
 'rain',
 'tell',
 'best',
 'let',
 'hear',
 'keep',
 'everyon',
 'lon

### Building Features

Now let's build the features. Using the top 5,000 words, create a 2-dimensional matrix to record whether each of those words is contained in each document (tweet). Then you also have an output column to indicate whether the sentiment in each tweet is positive. For example, assuming your bag of words has 5 items (`['one', 'two', 'three', 'four', 'five']`) out of 4 documents (`['A', 'B', 'C', 'D']`), your feature set is essentially:

| Doc | one | two | three | four | five | is_positive |
|---|---|---|---|---|---|---|
| A | True | False | False | True | False | True |
| B | False | False | False | True | True | False |
| C | False | True | False | False | False | True |
| D | True | False | False | False | True | False|

However, because the `nltk.NaiveBayesClassifier.train` class we will use in the next step does not work with Pandas dataframe, the structure of your feature set should be converted to the Python list looking like below:

```python
[
	({
		'one': True,
		'two': False,
		'three': False,
		'four': True,
		'five': False
	}, True),
	({
		'one': False,
		'two': False,
		'three': False,
		'four': True,
		'five': True
	}, False),
	({
		'one': False,
		'two': True,
		'three': False,
		'four': False,
		'five': False
	}, True),
	({
		'one': True,
		'two': False,
		'three': False,
		'four': False,
		'five': True
	}, False)
]
```

To help you in this step, watch the [following video](https://www.youtube.com/watch?v=-vVskDsHcVc) to learn how to build the feature set with Python and NLTK. The source code in this video can be found [here](https://pythonprogramming.net/words-as-features-nltk-tutorial/).

[![Building Features](building-features.jpg)](https://www.youtube.com/watch?v=-vVskDsHcVc)

In [45]:
# your code here
# Define a function to create feature dictionaries
def document_features(document, top_words):
    document_words = set(document)
    return {word: (word in document_words) for word in top_words}


In [46]:
# Create the feature set in the required format
featuresets = [
    (document_features(doc, top_5000_words), is_positive)
    for doc, is_positive in zip(df['text_processed'], df['target'])
]

### Building and Traininng Naive Bayes Model

In this step you will split your feature set into a training and a test set. Then you will create a Bayes classifier instance using `nltk.NaiveBayesClassifier.train` ([example](https://www.nltk.org/book/ch06.html)) to train with the training dataset.

After training the model, call `classifier.show_most_informative_features()` to inspect the most important features. The output will look like:

```
Most Informative Features
	    snow = True            False : True   =     34.3 : 1.0
	  easter = True            False : True   =     26.2 : 1.0
	 headach = True            False : True   =     20.9 : 1.0
	    argh = True            False : True   =     17.6 : 1.0
	unfortun = True            False : True   =     16.9 : 1.0
	    jona = True             True : False  =     16.2 : 1.0
	     ach = True            False : True   =     14.9 : 1.0
	     sad = True            False : True   =     13.0 : 1.0
	  parent = True            False : True   =     12.9 : 1.0
	  spring = True            False : True   =     12.7 : 1.0
```

The [following video](https://www.youtube.com/watch?v=rISOsUaTrO4) will help you complete this step. The source code in this video can be found [here](https://pythonprogramming.net/naive-bayes-classifier-nltk-tutorial/).

[![Building and Training NB](nb-model-building.jpg)](https://www.youtube.com/watch?v=rISOsUaTrO4)

In [49]:
# splt
import nltk
import random

# Shuffle the feature sets to randomize the order
random.shuffle(featuresets)

# Split the data into a training set and a test set (e.g., 80% training, 20% test)
split_index = int(0.8 * len(featuresets))
training_set = featuresets[:split_index]
test_set = featuresets[split_index:]

# Train the Naive Bayes classifier using the training set
classifier = nltk.NaiveBayesClassifier.train(training_set)

In [50]:
classifier.show_most_informative_features()

Most Informative Features
                   smile = True                4 : 0      =     27.1 : 1.0
                  cancel = True                0 : 4      =     19.4 : 1.0
                     sad = True                0 : 4      =     18.0 : 1.0
                    lone = True                0 : 4      =     16.7 : 1.0
                   worst = True                0 : 4      =     15.3 : 1.0
                    poor = True                0 : 4      =     13.4 : 1.0
                  welcom = True                4 : 0      =     13.3 : 1.0
                 stomach = True                0 : 4      =     12.6 : 1.0
                  inspir = True                4 : 0      =     12.1 : 1.0
                     vip = True                4 : 0      =     12.1 : 1.0


### Testing Naive Bayes Model

Now we'll test our classifier with the test dataset. This is done by calling `nltk.classify.accuracy(classifier, test)`.

As mentioned in one of the tutorial videos, a Naive Bayes model is considered OK if your accuracy score is over 0.6. If your accuracy score is over 0.7, you've done a great job!

In [52]:
# your code here
nltk.classify.accuracy(classifier, test_set)

0.728

## Bonus Question 1: Improve Model Performance

If you are still not exhausted so far and want to dig deeper, try to improve your classifier performance. There are many aspects you can dig into, for example:

* Improve stemming and lemmatization. Inspect your bag of words and the most important features. Are there any words you should furuther remove from analysis? You can append these words to further remove to the stop words list.

* Remember we only used the top 5,000 features to build model? Try using different numbers of top features. The bottom line is to use as few features as you can without compromising your model performance. The fewer features you select into your model, the faster your model is trained. Then you can use a larger sample size to improve your model accuracy score.

In [ ]:
# your code here

## Bonus Question 2: Machine Learning Pipeline

In a new Jupyter Notebook, combine all your codes into a function (or a class). Your new function will execute the complete machine learning pipeline job by receiving the dataset location and output the classifier. This will allow you to use your function to predict the sentiment of any tweet in real time. 

In [ ]:
# your code here

## Bonus Question 3: Apache Spark

If you have completed the Apache Spark advanced topic lab, what you can do is to migrate your pipeline from local to a Databricks Notebook. Share your notebook with your instructor and classmates to show off your achievements!

In [ ]:
# your code here